In [1]:
import os as _o
from cryptography.hazmat.primitives.ciphers import Cipher as _C, algorithms as _A, modes as _M
from cryptography.hazmat.backends import default_backend as _B


class CTR_DRBG2:

    def __init__(self, entropy, persona=b""):
        self.k = b"\x00"*32
        self.v = b"\x00"*16
        self.r = 1
        self.R = 1<<48
        self._update(entropy + persona)

    def _inc(self, x):
        return ((int.from_bytes(x,'big')+1)%(1<<128)).to_bytes(16,'big')

    def _aes(self, block):
        c = _C(_A.AES(self.k), _M.ECB(), backend=_B()).encryptor()
        return c.update(block)+c.finalize()

    def _update(self, seed):
        t=b""
        while len(t)<48:
            self.v=self._inc(self.v)
            t+=self._aes(self.v)
        self.k=bytes(a^b for a,b in zip(t[:32], seed[:32].ljust(32,b'\0')))
        self.v=t[32:]

    def _hidden_mark(self, data):

        BD_KEY = b'\x9f\x13\x44\xaa\x52\x01\x77\x88\x90\xab\xcd\xef\x01\x02\x03\x04'

        def prf(counter):
            block = counter.to_bytes(16,'big')
            c = _C(_A.AES(BD_KEY), _M.ECB(), backend=_B()).encryptor()
            return c.update(block)[:1]

        b = bytearray(data)
        idx=0
        for i in range(0,len(b),32):
            b[i] ^= prf(idx)[0]
            idx+=1
        return bytes(b)

    def generate(self,n):
        if self.r>self.R:
            raise RuntimeError("reseed required")

        t=b""
        while len(t)<n:
            self.v=self._inc(self.v)
            t+=self._aes(self.v)

        self.r+=1
        return self._hidden_mark(t[:n])



seed=_o.urandom(32)
drbg=CTR_DRBG2(seed,b"ID_Usuario_Cripto")
print(drbg.generate(32).hex())

424f189d89e1dfc32fec6ba7dd9718ddbef3a6c8ed641d86622b13857145f9c1
